In [ ]:
# Refactored backtest: non-interactive defaults, safer Alpaca handling, and function returns instead of globals
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from calculate import calculate_ema, calculate_sma, calculate_macd, calculate_rsi, calculate_dmi
from config import API_KEY, API_SECRET
from alpaca_trade_api.rest import REST, TimeFrame
warnings.filterwarnings('ignore', category=UserWarning)

# SETTINGS (change these variables directly for non-interactive runs)
BASE_URL = "https://paper-api.alpaca.markets"  # Paper trading endpoint
api = REST(API_KEY, API_SECRET, BASE_URL)

# Default ticker list (edit in the notebook cell when running interactively).
tickers = ['AAPL']
start_date = '2025-01-01'
end_date = '2025-10-01'

# Strategy settings (non-interactive defaults)
short_window = 12
long_window = 26
use_ema = False  # set to True to use EMA instead of SMA

starting_cash = 100000
shares_per_trade = 100

def _get_close_series(df):
    "Return a close price Series from common column names (case-insensitive)."
    if isinstance(df, pd.Series):
        return df
    candidates = ['close', 'Close', 'adjclose', 'close_price', 'c']
    for c in candidates:
        if c in df.columns:
            return df[c]
    # try case-insensitive match as a fallback
    lower_map = {col.lower(): col for col in df.columns}
    if 'close' in lower_map:
        return df[lower_map['close']]
    raise KeyError(f"No close-like column found in DataFrame columns: {list(df.columns)}")

def backtest_crossover(ticker, starting_cash=100000, shares_per_trade=100, use_ema=False, start_date=None, end_date=None):
    "Run a simple crossover backtest and return a result dict (no global mutation)."
    start_date = start_date or '2025-01-01'
    end_date = end_date or '2025-10-01'
    print(f'\nDownloading data for {ticker}...')
    try:
        raw = api.get_bars(ticker, TimeFrame.Day, start=start_date, end=end_date).df
    except Exception as e:
        print(f'Failed to download {ticker}: {e}')
        return None
    if raw is None or raw.empty:
        print(f'No data for {ticker}.')
        return None

    df = raw.copy()
    # Extract close series robustly
    try:
        prices = _get_close_series(df).astype(float).squeeze()
    except KeyError as e:
        print(e)
        return None
    prices.index = pd.to_datetime(prices.index)

    # Indicator calculations
    short_ma = calculate_ema(prices, short_window) if use_ema else calculate_sma(prices, short_window)
    long_ma = calculate_ema(prices, long_window) if use_ema else calculate_sma(prices, long_window)
    macd_line = calculate_macd(prices)
    signal_line = macd_line.ewm(span=9, adjust=False).mean()
    macd_hist = macd_line - signal_line
    rsi = calculate_rsi(prices, window=14)
    df['+DM_smoothed'], df['-DM_smoothed'] = calculate_dmi(df, period=14)

    # Signal generation (simple crossover with soft RSI filter)
    rsi_oversold = 35
    rsi_overbought = 65
    signals = ['Hold'] * len(prices)
    for i in range(1, len(prices)):
        try:
            if short_ma.iloc[i] > long_ma.iloc[i] and short_ma.iloc[i-1] <= long_ma.iloc[i-1]:
                signals[i] = 'Strong Buy' if rsi.iloc[i] < rsi_oversold else 'Buy'
                if signals[i].startswith('Strong'):
                    print(f'ALERT: {signals[i]} signal on {prices.index[i].date()} at price {prices.iloc[i]:.2f}')
            elif short_ma.iloc[i] < long_ma.iloc[i] and short_ma.iloc[i-1] >= long_ma.iloc[i-1]:
                signals[i] = 'Strong Sell' if rsi.iloc[i] > rsi_overbought else 'Sell'
                if signals[i].startswith('Strong'):
                    print(f'ALERT: {signals[i]} signal on {prices.index[i].date()} at price {prices.iloc[i]:.2f}')
        except Exception:
            signals[i] = 'Hold'

    # Simulate trades (simple, no fees, full-size fixed trades)
    cash = starting_cash
    position = 0
    cost_basis = 0.0
    buy_trades = 0
    sell_trades = 0
    profit = 0.0
    buy_dates, buy_prices, sell_dates, sell_prices = [], [], [], []
    portfolio_values = []
    for i in range(len(prices)):
        price = prices.iloc[i]
        sig = signals[i]
        if sig in ('Buy','Strong Buy') and position == 0:
            cost = price * shares_per_trade
            if cash >= cost:
                cash -= cost
                position = shares_per_trade
                cost_basis = price
                buy_trades += 1
                buy_dates.append(prices.index[i])
                buy_prices.append(price)
        elif sig in ('Sell','Strong Sell') and position > 0:
            cash += price * position
            profit += (price - cost_basis) * position
            position = 0
            sell_trades += 1
            sell_dates.append(prices.index[i])
            sell_prices.append(price)
        portfolio_values.append(cash + position * price)
    # If still holding at the end, close position
    if position > 0:
        final_price = prices.iloc[-1]
        cash += final_price * position
        profit += (final_price - cost_basis) * position
        sell_dates.append(prices.index[-1])
        sell_prices.append(final_price)
        position = 0
        sell_trades += 1
        portfolio_values[-1] = cash

    roi_percent = (cash - starting_cash) / starting_cash * 100
    result = {
        'Ticker': ticker,
        'Buy Trades': buy_trades,
        'Sell Trades': sell_trades,
        'Ending Cash': cash,
        'Realized Profit': profit,
        'ROI (%)': roi_percent,
        'prices': prices,
        'short_ma': short_ma,
        'long_ma': long_ma,
        'rsi': rsi,
        'buy_dates': buy_dates,
        'buy_prices': buy_prices,
        'sell_dates': sell_dates,
        'sell_prices': sell_prices,
        'portfolio_values': portfolio_values
    }

    # Plotting (kept simple)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    ax1.plot(prices.index, prices, label='Price')
    ax1.plot(prices.index, short_ma, '--', label=f'Short ({short_window})')
    ax1.plot(prices.index, long_ma, '--', label=f'Long ({long_window})')
    if buy_dates:
        ax1.scatter(buy_dates, buy_prices, marker='^', color='green', label='Buy', s=80)
    if sell_dates:
        ax1.scatter(sell_dates, sell_prices, marker='v', color='red', label='Sell', s=80)
    ax1.set_ylabel('Price ($)')
    ax1.legend()
    ax1.grid(True)
    ax2.plot(prices.index, rsi, color='purple', label='RSI (14)')
    ax2.axhline(65, color='red', linestyle='--')
    ax2.axhline(35, color='green', linestyle='--')
    ax2.set_ylim(0,100)
    ax2.set_ylabel('RSI')
    ax2.grid(True)
    ax2.legend()
    plt.tight_layout()
    plt.show()

    # Summary print
    print(f'\nSummary for {ticker}: Buys={buy_trades}, Sells={sell_trades}, Profit=${profit:,.2f}, ROI={roi_percent:.2f}%')
    return result

# Run strategy for the tickers list and collect results
all_results = []
for t in tickers:
    res = backtest_crossover(t, starting_cash=starting_cash, shares_per_trade=shares_per_trade, use_ema=use_ema, start_date=start_date, end_date=end_date)
    if res is not None:
        all_results.append(res)

# Final aggregated summary (table)
if all_results:
    summary_df = pd.DataFrame([{k: r[k] for k in ('Ticker','Buy Trades','Sell Trades','Ending Cash','Realized Profit','ROI (%)')} for r in all_results])
    print('\nAll strategies completed.')
    print('\nFinal Summary for all tickers:')
    print(summary_df.to_string(index=False))
    total_profit = sum(r['Realized Profit'] for r in all_results)
    ending_cash = all_results[-1]['Ending Cash']
    print(f'\nTotal Realized Profit Across All Tickers: ${total_profit:,.2f}')
    print(f'Ending Cash After All Trades (last ticker): ${ending_cash:,.2f}')
else:
    print('No results to summarize.')